## A  Simple Cast Study to Understand about Robust Optimization in Process Optimzation

In [104]:
from pyomo.environ import *
import os

### Problem Description

Imagine a process with two reactions when written in Mass Balance:

$$ A \longrightarrow 0.4 C + 0.6 D \quad \quad \text{RXN1}$$
$$ B \longrightarrow 0.7 C + 0.3 D \quad \quad \text{RXN2}$$

The two reactions happen together in the reactor, and the product stream goes to a separator that performs a complete binary separation.

![Optional Alt Text](illustrations/toy.png)

Nomenclature

Sets

\begin{align*}
    & I && \quad \text{Set of Streams: \{1,2,3,4,5,6\}} \\
    & J && \quad \text{Set of Species: \{A,B,C,D\}} \\
    & T && \quad \text{Set of Technologies: \{R1, R2, SEP1\}} \\
\end{align*}

Parameters

\begin{align*}
    & Q^{Load}_{j} && \quad \text{Mass flow rate of stream i for species j} \\
    & \theta_{t,j} \quad  t \in \{R1, R2\} && \quad \text{Conversion of technology t for species j} \\
    & \alpha_{t} && \quad \text{Cost coefficient of technology t} \\
    & \beta_{j} && \quad \text{Price of species j}
\end{align*}

Variables

\begin{align*}
    & F_{i,j} \quad \text{Mass flow rate of stream i for species j} \\
    & W_{t} \quad \text{Capacity of technology t} \\
    & A_{t} \quad \text{Cost of technology t} \\
    & B_{j} \quad \text{Cost of species, Negative for products}
\end{align*}


### Feed Constraints

First we begin with total supply of each feedstock we have.

\begin{align*}
    & F_{1, A} \le Q^{Load}_{A} \\
    & F_{2, B} \le Q^{Load}_{B} \\
\end{align*}

We also requires the feed to be pure streams:

\begin{align*}
    & F_{1, j} = 0 \text{   if } j \neq A \\
    & F_{2, j} = 0 \text{   if } j \neq B \\
\end{align*}


### Mass Balances

For the reactions, we have

\begin{align*}
    & F_{3, j} - F_{1,j} = \theta_{R1,j} F_{1,j} \\
\end{align*}

\begin{align*}
    & F_{4, j} - F_{2,j} = \theta_{R2,j} F_{2,j} \\
\end{align*}

For the Separation:

\begin{align*}
    & F_{3, j} + F_{4,j} = F_{5,j} \text{   if } j = C \\
\end{align*}

\begin{align*}
    & F_{3, j} + F_{4,j} = F_{6,j} \text{   if } j \neq C \\
\end{align*}

This completes our mass balance.

### Design Feasibility

For the capacities:

\begin{align*}
    & F_{1, A} \le W_{R1} \\
    & F_{2, B} \le W_{R2} \\
    & \sum_{j} (F_{3, j} + F_{4, j}) \le W_{R3} \\
\end{align*}

This completes the design feasibility  constraints if we only consider the capacity.

### Economics

In this case, we assume a linear relationship between both capital and operating cost associated with technologies.

\begin{align*}
    & A_{t} = \alpha_{t} W_{t}  \\
\end{align*}

As for the material cost/sales:

\begin{align*}
    & B_{A} = \beta_{A} F_{1,A}  \\
    & B_{B} = \beta_{B} F_{2,B}  \\
    & B_{C} = \beta_{C} F_{5,C}  \\
    & B_{D} = \beta_{D} F_{6,D}  \\
\end{align*}


### Deterministic Model


$$  Min_{\{F_{1,A}, F_{2,B}\}} \sum_{t \in T} W_{t} + \sum_{j \in J} P_{j}  $$

subject to

\begin{align*}
    & F_{1, A} \le Q^{Load}_{A} \\
    & F_{2, B} \le Q^{Load}_{B} \\
    & F_{1, j} = 0 \text{   if } j \neq A \\
    & F_{2, j} = 0 \text{   if } j \neq B \\
    & F_{3, j} - F_{1,j} = \theta_{R1,j} F_{1,j} \\
    & F_{4, j} - F_{2,j} = \theta_{R2,j} F_{2,j} \\
    & F_{3, j} + F_{4,j} = F_{5,j} \text{   if } j = C \\
    & F_{3, j} + F_{4,j} = F_{6,j} \text{   if } j \neq C \\
    & F_{1, A} \le W_{R1} \\
    & F_{2, B} \le W_{R2} \\
    & \sum_{j} (F_{3, j} + F_{4, j}) \le W_{R3} \\
    & W_{R1} = \alpha_{R1} F_{1,A}  \\
    & W_{R2} = \alpha_{R2} F_{2,B}  \\
    & W_{SEP1} = \alpha_{R3} \sum_{j}(F_{3,j} + F_{4,j}) \\
    & P_{A} = \beta_{A} F_{1,A}  \\
    & P_{B} = \beta_{B} F_{2,B}  \\
    & P_{C} = \beta_{C} F_{5,C}  \\
    & P_{D} = \beta_{D} F_{6,D}  \\
\end{align*}


In [105]:
def build_DM():
    m = ConcreteModel()

    m.i = Set(initialize=[1,2,3,4,5,6], doc="streams")
    m.j = Set(initialize=["A","B","C","D"])
    m.t = Set(initialize=["R1","R2","SEP1"])

    m.Qsup = Param(m.j,initialize={"A": 100, "B": 100})
    m.theta = Param(m.t,m.j,initialize={("R1","A"): -1, ("R1","B"): 0, ("R1","C"): 0.4, ("R1","D"): 0.6,
                                        ("R2","A"): 0, ("R2","B"): -1, ("R2","C"): 0.7, ("R2","D"): 0.3,
                                        })
    m.alpha = Param(m.t,initialize={"R1": 1, "R2": 1.5, "SEP1": 2})
    m.beta = Param(m.j,initialize={"A": 5, "B": 7, "C": -20, "D": -30})

    m.F = Var(m.i, m.j, domain=NonNegativeReals, doc="Flow rate of stream i speices j")
    m.W = Var(m.t, domain=NonNegativeReals, doc="Technology Capacity")
    m.A = Var(m.t, domain=NonNegativeReals, doc="Technology Cost")
    m.B = Var(m.j, domain=NonNegativeReals, doc="Material Cost")

    def _csupply(m, j):
        if not j in {"A","B"}:
            return Constraint.Skip
        if j == "A":
            return m.F[1,j] <= m.Qsup[j]
        elif j == "B":
            return m.F[2,j] <= m.Qsup[j]
        raise IndexError
    m.csupply = Constraint(m.j, rule=_csupply)

    def _cfeed(m, i, j):
        if not i in {1,2}:
            return Constraint.Skip
        if not (i,j) in {(1,"A"), (2,"B")}:
            return m.F[i,j] == 0
        return Constraint.Skip
    m.cfeed = Constraint(m.i, m.j, rule=_cfeed)

    def _cR1(m, j):
        return m.F[3,j] - m.F[1,j] == m.theta["R1",j] * m.F[1,"A"]
    m.cR1 = Constraint(m.j, rule=_cR1)

    def _cR2(m, j):
        return m.F[4,j] - m.F[2,j] == m.theta["R2",j] * m.F[2,"B"]
    m.cR2 = Constraint(m.j, rule=_cR2)

    def _csep(m, j):
        if not j in {"D"} and j == 5:
            return m.F[3,j] + m.F[4,j] == m.F[5,j]
        return m.F[3,j] + m.F[4,j] ==  m.F[6,j]
    m.csep = Constraint(m.j, rule=_csep)

    def _csep2(m, j):
        return m.F[3,j] + m.F[4,j] == m.F[5,j] + m.F[6,j]
    m.csep2 = Constraint(m.j, rule=_csep2)

    def _ccap(m, t):
        if t == "R1":
            return m.W[t] >= m.F[1,"A"]
        elif t == "R2":
            return m.W[t] >= m.F[2,"B"]
        elif t == "SEP1":
            return m.W[t] >= sum(m.F[3,j] + m.F[4,j] for j in m.j)
        raise IndexError
    m.ccap = Constraint(m.t, rule=_ccap)

    def _techcost(m, t):
        return m.alpha[t] * m.W[t] == m.A[t]
    m.techcost = Constraint(m.t, rule=_techcost)

    def _matcost(m, j):
        if j == "A":
            return m.beta[j] * m.F[1, j] == m.B[j]
        elif j == "B":
            return m.beta[j] * m.F[2, j] == m.B[j]
        elif j == "C":
            return m.beta[j] * m.F[5, j] == m.B[j]
        elif j == "D":
            return m.beta[j] * m.F[6, j] == m.B[j]
        raise IndexError
    m.matcost = Constraint(m.j, rule=_matcost)

    m.obj = Objective(expr=sum(m.A[t] for t in m.t) + sum(m.B[j] for j in m.j),  sense=minimize)

    return m

m1 = build_DM()

In [106]:
def solve_with_gams(m):
    opt1 = SolverFactory('gams')
    io_options = dict()

    io_options['solver'] = "cplex"
    res1 = opt1.solve(m,
        tee=True,
        add_options = ['option reslim=1000; option optcr=0.0;'],
        io_options=io_options)
    return res1

def solve_with_gurobi(m):
    opt1 = SolverFactory("gurobi_persistent")
    opt1.set_instance(m)
    res1 = opt1.solve()
    return res1

In [107]:
m1.F[1, "A"].fix(50)
m1.F[2, "B"].fix(50)
solve_with_gams(m1)
m1.write("m1.lp", io_options={'symbolic_solver_labels': True})

--- Job model.gms Start 12/01/23 19:42:32 43.2.0 859d62d5 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\GAMS\43\gmsprmNT.txt
    C:\Users\justi\OneDrive\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    Input C:\Users\justi\AppData\Local\Temp\tmpf2xajqap\model.gms
    Output C:\Users\justi\AppData\Local\Temp\tmpf2xajqap\output.lst
    ScrDir C:\Users\justi\AppData\Local\Temp\tmpf2xajqap\225a\
    SysDir C:\GAMS\43\
    CurDir C:\Users\justi\AppData\Local\Temp\tmpf2xajqap\
    LogOption 3
Licensee: Small MUD - 5 User License                     G221121|0002AP-GEN
          University of Delaware, Chemical and Biomolecular EngineeriDC3967
          C:\GAMS\43\gamslice.txt
          License Admin: Marianthi Ierapetritou, mgi@udel.edu              
Processor information: 1 socket(s), 10 core(s), and 20 thread(s) available
GAMS 43.2.0   Copyright (C) 1987-2023 GAMS Development. All rights reserved
--- Starting compilation
--- model.gms(251) 2 Mb
--- Starting execution: elap

('m1.lp', 2414978276880)

In [108]:
m1.A.pprint(), m1.B.pprint()

A : Technology Cost
    Size=3, Index=t
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
      R1 :     0 :   0.0 :  None : False : False : NonNegativeReals
      R2 :     0 :   0.0 :  None : False : False : NonNegativeReals
    SEP1 :     0 :   0.0 :  None : False : False : NonNegativeReals
B : Material Cost
    Size=4, Index=j
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      A :     0 :   0.0 :  None : False : False : NonNegativeReals
      B :     0 :   0.0 :  None : False : False : NonNegativeReals
      C :     0 :   0.0 :  None : False : False : NonNegativeReals
      D :     0 :   0.0 :  None : False : False : NonNegativeReals


(None, None)